<a href="https://colab.research.google.com/github/beacon404/langchain-mailgen/blob/main/langchain_EmailGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
pip install langchain pypdf langchain-community google-generativeai langchain-google-genai unstructured langchain_google_genai markdown

In [137]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI
import google.generativeai as genai
from google.colab import userdata
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import markdown

In [ ]:
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-1.5-pro')
response = model.generate_content("what is lainchain?")
print(response.text)


"Lainchain" doesn't seem to be a real term or technology. It's possible there's a typo, and you meant to ask about **"Langchain."** 

**Langchain** is a powerful framework designed specifically for developing applications powered by language models like ChatGPT. 

Here's a breakdown of what makes Langchain useful:

* **Simplifies Complex Interactions:** Langchain makes it easier to build applications that go beyond simple question-and-answer interactions with language models. 
* **Chain Creation:** You can create "chains" of commands and prompts, allowing for more sophisticated use cases.  For example, you could have a chain that first summarizes a long document and then answers questions based on that summary.
* **Integration with Other Tools:** Langchain seamlessly integrates with other tools and services, including:
    * **Data sources:** Access and use data from APIs, databases, and documents.
    * **Other language models:** You're not limited to just one—incorporate different mo

In [83]:
def secondary_company_data_scrap(company_name):
  url = f"https://pu.edu.np/about-us/general-information/"
  print(url)
  loader = UnstructuredURLLoader(urls=[url])
  print(loader)
  print(f"\n {loader.load}")
  return loader.load()

# extract all the text from the given url
data = secondary_company_data_scrap("")
print(data)

print (f"You have {len(data)} document(s)")

https://pu.edu.np/about-us/general-information/

 <bound method UnstructuredURLLoader.load of <langchain_community.document_loaders.url.UnstructuredURLLoader object at 0x7e90c45958d0>>
[Document(metadata={'source': 'https://pu.edu.np/about-us/general-information/'}, page_content='Search for:\n\nSearch for:\n\nAPPLY ONLINE FOR DEGREE CERTIFICATE\n\nPUBLICATIONS\n\nSCHOLARSHIP\n\nLIBRARY\n\nSTUDENT\n\nLMS(SOE)\n\nICT\n\nLOGIN\n\nAbout Us\n\nGeneral InformationMessage from VCUniversity Service CommissionGovernancePlanning & Program DivisionPokhara University Teaching Hospital\n\nAcademics\n\nFacultiesSchools & CollegesProgramsCurriculum Development Center\n\nAdmission\n\nUndergraduateGraduateMPhilPhD\n\nResearch\n\nExamination\n\nExamination PortalGeneral InformationNoticesResultsAcademic SystemAcademic CalendarConvocationContact Us\n\nGlobal\n\nInternational CenterAurora ProjectFAB Erasmus+ ProjecteAccess ProjectPartner InstitutionsMoU Latest News & HeadlinesPhoto Gallery\n\nAbout Us\n\n

In [80]:
print (f"Preview of your data:\n\n{data[0].page_content}")

Preview of your data:

Search for:

Search for:

APPLY ONLINE FOR DEGREE CERTIFICATE

PUBLICATIONS

SCHOLARSHIP

LIBRARY

STUDENT

LMS(SOE)

ICT

LOGIN

About Us

General InformationMessage from VCUniversity Service CommissionGovernancePlanning & Program DivisionPokhara University Teaching Hospital

Academics

FacultiesSchools & CollegesProgramsCurriculum Development Center

Admission

UndergraduateGraduateMPhilPhD

Research

Examination

Examination PortalGeneral InformationNoticesResultsAcademic SystemAcademic CalendarConvocationContact Us

Global

International CenterAurora ProjectFAB Erasmus+ ProjecteAccess ProjectPartner InstitutionsMoU Latest News & HeadlinesPhoto Gallery

About Us

General InformationMessage from VCUniversity Service CommissionGovernancePlanning & Program DivisionPokhara University Teaching Hospital

Academics

FacultiesSchools & CollegesProgramsCurriculum Development Center

Admission

UndergraduateGraduateMPhilPhD

Research

Examination

Examination PortalGene

In [123]:
split_text = RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 0
)
chunks = split_text.split_documents(data)
print(chunks[0])
print (f"You now have {len(chunks)} documents")

page_content='Search for:

Search for:

APPLY ONLINE FOR DEGREE CERTIFICATE

PUBLICATIONS

SCHOLARSHIP

LIBRARY

STUDENT

LMS(SOE)

ICT

LOGIN

About Us

General InformationMessage from VCUniversity Service CommissionGovernancePlanning & Program DivisionPokhara University Teaching Hospital

Academics

FacultiesSchools & CollegesProgramsCurriculum Development Center

Admission

UndergraduateGraduateMPhilPhD

Research

Examination

Examination PortalGeneral InformationNoticesResultsAcademic SystemAcademic CalendarConvocationContact Us

Global

International CenterAurora ProjectFAB Erasmus+ ProjecteAccess ProjectPartner InstitutionsMoU Latest News & HeadlinesPhoto Gallery

About Us' metadata={'source': 'https://pu.edu.np/about-us/general-information/'}
You now have 11 documents


In [124]:
summary_map_prompt = """Below is a section of a website about {secondary_company}

Write a concise summary about {secondary_company}. If the information is not about {secondary_company}, exclude it from your summary.

{text}

% CONCISE SUMMARY:"""

summary_map_prompt_templete = PromptTemplate(template=summary_map_prompt, input_variables=["text", "secondary_company"])


In [125]:
combine_prompt = """
Your goal is to write a personalized outbound email from {sender}, a president at {company} to {secondary_company}.

A good email is personalized and combines information about the two companies on how they can help each other.
Be sure to use value selling: A sales methodology that focuses on how your product or service will provide value to the customer instead of focusing on price or solution.

% INFORMATION ABOUT {company}:
{company_information}

% INFORMATION ABOUT {secondary_company}:
{text}

% INCLUDE THE FOLLOWING PIECES IN YOUR RESPONSE:
- Start the email with the sentence: "We love that {secondary_company} helps teams..." then insert what they help teams do.
- The sentence: "We can help you do XYZ by ABC" Replace XYZ with what {secondary_company} does and ABC with what {company} does
- A 1-2 sentence description about {company}, be brief
- End your email with a call-to-action such as asking them to set up time to talk more

% YOUR RESPONSE:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["sender", "company", "secondary_company", \
                                                                         "text", "company_information"])

In [126]:
company_information = """
Overview

    Established: 1997, making it the oldest student club at Kathmandu University.
    Focus: Promoting ICT through various activities and fostering a community of tech enthusiasts.
    Structure: Includes sub-communities focused on specific areas like open source, entrepreneurship, design, cybersecurity, etc.
    Impact: Has organized over 120 events, conducted 500+ workshops, and has a membership of over 1000.

Activities and Goals

    Core activities: Organizing events like IT Meet, celebrating Software Freedom Day, and conducting regular workshops.
    Community building: Creating a platform for students to network, share knowledge, and collaborate on projects.
    Skill development: Offering workshops and training programs to enhance technical and soft skills.
    Promoting ICT: Raising awareness about ICT and encouraging innovation through various initiatives.

Additional Information

    Open Source Focus: KUCC has a dedicated open source community (KUOSC) promoting FOSS.
    Diverse Interests: The club caters to a wide range of interests with specialized communities.
    Impactful Reach: With a large membership and active engagement, KUCC has a significant influence on the student community.
"""

In [127]:
llm = ChatGoogleGenerativeAI(temperature=.7, google_api_key=userdata.get("GOOGLE_API_KEY"), model="gemini-1.5-pro")

chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=summary_map_prompt_templete,
                             combine_prompt=combine_prompt_template,
                             verbose=True
                            )


print(llm)
print(f"\n{chain}")

model='models/gemini-1.5-pro' google_api_key=SecretStr('**********') client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7e90c45927a0> async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7e90c45757e0> default_metadata=()

verbose=True llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['secondary_company', 'text'], template='Below is a section of a website about {secondary_company}\n\nWrite a concise summary about {secondary_company}. If the information is not about {secondary_company}, exclude it from your summary.\n\n{text}\n\n% CONCISE SUMMARY:'), llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7e90c45927a0>, async_client=<google.ai.generativelanguage_v1beta.se

In [128]:
output = chain({"input_documents": chunks, # The seven docs that were created before
                "company": "Kathmandu University Computer Club", \
                "company_information" : company_information,
                "sender" : "Sanjay Pahari", \
                "secondary_company" : "Pokhara University"
               })



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is a section of a website about Pokhara University

Write a concise summary about Pokhara University. If the information is not about Pokhara University, exclude it from your summary.

Search for:

Search for:

APPLY ONLINE FOR DEGREE CERTIFICATE

PUBLICATIONS

SCHOLARSHIP

LIBRARY

STUDENT

LMS(SOE)

ICT

LOGIN

About Us

General InformationMessage from VCUniversity Service CommissionGovernancePlanning & Program DivisionPokhara University Teaching Hospital

Academics

FacultiesSchools & CollegesProgramsCurriculum Development Center

Admission

UndergraduateGraduateMPhilPhD

Research

Examination

Examination PortalGeneral InformationNoticesResultsAcademic SystemAcademic CalendarConvocationContact Us

Global

International CenterAurora ProjectFAB Erasmus+ ProjecteAccess ProjectPartner InstitutionsMoU Latest News & HeadlinesPhoto Gallery

About Us

% CONCISE SUMMARY:
Promp

In [146]:
email_markdown = output['output_text']
print(email_markdown)
# print(markdown.markdown(email_markdown))

Subject: Kathmandu University Computer Club: Partnering for Student Success

Dear Pokhara University Team,

We love that Pokhara University helps teams **develop critical thinking skills, a positive work ethic, and enhanced competitiveness to succeed in their careers and contribute to the global community**. 

At Kathmandu University Computer Club (KUCC), we share your passion for empowering students with the skills they need to thrive in the tech-driven world. We can help you **further enhance student's technical skills and prepare them for the job market** by **offering specialized workshops and training programs led by industry experts and experienced members of our club**. 

As the oldest student club at Kathmandu University, KUCC is dedicated to promoting ICT and fostering a community of over 1000 tech enthusiasts through events, workshops, and collaborative projects. 

We'd love to explore how a partnership could benefit both our student bodies. Would you be open to a brief call 